# MT Bias Estimation of the Actual Flip Angle $B_1$-Mapping Method
This notebooks estimates the bias of $B_1$ caused by the magnetization transfer effect in the [actual flip angle method](https://doi.org/10.1002/mrm.21120). 

First, we load all required packages:

In [1]:
using MRIgeneralizedBloch
using StaticArrays

Define the point in parameter space at which the bias will be calculated:

In [2]:
m0s = 0.1
R1a = 0.625 # 1/s
R1f = R1a   # 1/s
R1s = R1a   # 1/s
R2f = 15    # 1/s
Rx  = 30    # 1/s
T2s = 10e-6 # s
R2slT = precompute_R2sl()
ω0 = 0      # rad/s
B1 = 1;     # in units of B1_nominal

In the following cell, we define the sequence parameters as suggested in [Yarnykh 2016](https://doi.org/10.1002/mrm.25811). Replace with `TR1 = 20e-3` and `TR2 = 100e-3` to replicate the settings in [Yarnykh's original AFI publication](https://doi.org/10.1002/mrm.21120) or to `TR1 = 30e-3` and `TR2 = 150e-3` to replicate the settings [Yarnykh 2012](https://doi.org/10.1002/mrm.23224). 

In [3]:
α_nom = 60 * pi/180
TRF = 0.4e-3 # s

TR1 = 40e-3  # s
TR2 = 160e-3; # s

Simulate the steady state magnetization in each $T_R$. This is done calculating the propagator matrices `u_sp` for the (perfect) spoiler, `u_exc` for the excitation pulse, and `u_TR1` and `u_TR2` for free precession during respective $T_R$:

In [4]:
R2sl = R2slT[1](TRF, α_nom, B1, T2s)
u_sp =
    @SMatrix [
        0 0 0 0 0 0;
        0 0 0 0 0 0;
        0 0 1 0 0 0;
        0 0 0 0 0 0;
        0 0 0 0 1 0;
        0 0 0 0 0 1]

u_exc = exp(hamiltonian_linear(α_nom/TRF, B1, ω0,   TRF, m0s, R1f, R2f, Rx, R1s, R2sl))
u_TR1 = exp(hamiltonian_linear(0, B1, ω0, TR1 - TRF, m0s, R1f, R2f, Rx, R1s, R2sl))
u_TR2 = exp(hamiltonian_linear(0, B1, ω0, TR2 - TRF, m0s, R1f, R2f, Rx, R1s, R2sl));

The steady state magnetization is is given by solving the linear equation $m_{ss} = U m_{ss}$ or, more precisely, by solving $U m_{ss} - A_0 m_{ss} = C$, where $A_0$ is a unity matrix modified to have `0` as the last diagonal matrix and $C$ is a zero vector modified to have `1` as the last entry. 

In [5]:
U =  u_exc * u_sp * u_TR2 * u_exc * u_sp * u_TR1
Q = U - MRIgeneralizedBloch.A0(nothing)
m1 = Q \ MRIgeneralizedBloch.C(nothing)

U =  u_exc * u_sp * u_TR1 * u_exc * u_sp * u_TR2
Q = U - MRIgeneralizedBloch.A0(nothing)
m2 = Q \ MRIgeneralizedBloch.C(nothing);

Given the magnetization, we can estimate the flip angle `α_est` according to [Yarnykh's original AFI publication](https://doi.org/10.1002/mrm.21120):

In [6]:
n = TR2/TR1
r = m2[1] / m1[1]
α_est = acos((r*n - 1) / (n - r))

1.0183755610699896

And calculate the deviations from the nominal flip angle:

In [7]:
1 - α_est / (B1 * α_nom)

0.02752297319037278